# How to add human-in-the-loop processes to a ReAct agent (Functional API)

!!! info "Prerequisites"
    This guide assumes familiarity with the following:

    - Implementing [human-in-the-loop](../../concepts/human_in_the_loop) workflows with [interrupt](../../concepts/human_in_the_loop/#interrupt)
    - [How to create a ReAct agent using the Functional API](react-agent-from-scratch-functional.ipynb)

This guide demonstrates how to implement human-in-the-loop workflows in a ReAct agent using the LangGraph [Functional API](../../concepts/functional_api).

We will build off of the agent created in the [How to create a ReAct agent using the Functional API](react-agent-from-scratch-functional.ipynb) guide.

Specifically, we will demonstrate:

1. [How to allow the model to reach out to a human for assistance](#reach-out-to-a-human-for-assistance)
2. [How to review tool calls](#review-tool-calls-before-execution)

Both of these goals can be accomplished through use of the [interrupt](../../concepts/human_in_the_loop/#interrupt) function at key points in our application.

## Setup

First, let's install the required packages and set our API keys:

In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

<div class="admonition tip">
     <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for better debugging</p>
     <p style="padding-top: 5px;">
         Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM aps built with LangGraph — read more about how to get started in the <a href="https://docs.smith.langchain.com">docs</a>. 
     </p>
 </div>

### Define model and tools

Let's first define the tools and model we will use for our example. As in the [ReAct agent guide](react-agent-from-scratch-functional.ipynb), we will use a single place-holder tool that gets a description of the weather for a location.

We will use an [OpenAI](https://python.langchain.com/docs/integrations/providers/openai/) chat model for this example, but any model [supporting tool-calling](https://python.langchain.com/docs/integrations/chat/) will suffice.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

model = ChatOpenAI(model="gpt-4o-mini")


@tool
def get_weather(location: str):
    """Call to get the weather from a specific location."""
    # This is a placeholder for the actual implementation
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny!"
    elif "boston" in location.lower():
        return "It's rainy!"
    else:
        return f"I am not sure what the weather is in {location}"

In [2]:
# @task
# def call_tool(proposed_tool_call):
#     review = review_tool_call(proposed_tool_call)
#     if isinstance(review, ToolMessage):
#         return review
#     else:
#         tool_call = review
#         tool = tools_by_name[tool_call["name"]]
#         observation = tool.invoke(tool_call["args"])
#         return ToolMessage(content=observation, tool_call_id=tool_call["id"])

## Reach out to a human for assistance

To reach out to a human for assistance, we can simply add a tool that calls [interrupt](../../concepts/human_in_the_loop/#interrupt):

In [3]:
from langgraph.types import Command, interrupt

@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]


tools = [get_weather, human_assistance]

### Define tasks

Our tasks are otherwise unchanged from the [ReAct agent guide](react-agent-from-scratch-functional.ipynb):

1. **Call model**: We want to query our chat model with a list of messages.
2. **Call tool**: If our model generates tool calls, we want to execute them.

We just have one more tool accessible to the model.

In [4]:
from langchain_core.messages import ToolMessage
from langgraph.func import entrypoint, task

tools_by_name = {tool.name: tool for tool in tools}


@task
def call_model(messages):
    """Call model with a sequence of messages."""
    response = model.bind_tools(tools).invoke(messages)
    return response


@task
def call_tool(tool_call):
    tool = tools_by_name[tool_call["name"]]
    observation = tool.invoke(tool_call["args"])
    return ToolMessage(content=observation, tool_call_id=tool_call["id"])

### Define entrypoint

Our [entrypoint](../../concepts/functional_api/#entrypoint) is also unchanged from the [ReAct agent guide](react-agent-from-scratch-functional.ipynb):

In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages

checkpointer = MemorySaver()

@entrypoint(checkpointer=checkpointer)
def agent(messages, previous):
    if previous is not None:
        messages = add_messages(previous, messages)

    llm_response = call_model(messages).result()
    while True:
        if not llm_response.tool_calls:
            break

        # Execute tools
        tool_result_futures = [
            call_tool(tool_call) for tool_call in llm_response.tool_calls
        ]
        tool_results = [fut.result() for fut in tool_result_futures]

        # Append to message list
        messages = add_messages(messages, [llm_response, *tool_results])

        # Call model again
        llm_response = call_model(messages).result()

    # Generate final response
    messages = add_messages(messages, llm_response)
    return messages

### Usage

Let's invoke our model with a question that requires human assistance. Our question will also require an invocation of the `get_weather` tool:

In [6]:
def _print_step(step: dict) -> None:
    for task_name, result in step.items():
        if task_name in ("agent", "__interrupt__"):
            continue  # Just print task updates
        else:
            print(f"\n{task_name}:")
            result.pretty_print()

In [7]:
config = {"configurable": {"thread_id": "1"}}

In [8]:
user_message = {
    "role": "user",
    "content": (
        "Can you reach out for human assistance: what should I feed my cat? "
        "Separately, can you check the weather in San Francisco?"
    ),
}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)

{'role': 'user', 'content': 'Can you reach out for human assistance: what should I feed my cat? Separately, can you check the weather in San Francisco?'}

call_model:
================================== Ai Message ==================================
Tool Calls:
  human_assistance (call_Er3DsMcpdoO9yRy2NZHXV6mH)
 Call ID: call_Er3DsMcpdoO9yRy2NZHXV6mH
  Args:
    query: What should I feed my cat?
  get_weather (call_vkBLyCTkmGR2vMndJyxnfzn7)
 Call ID: call_vkBLyCTkmGR2vMndJyxnfzn7
  Args:
    location: San Francisco

call_tool:
================================= Tool Message =================================

It's sunny!


Note that we generate two tool calls, and although our run is interrupted, we did not block the execution of the `get_weather` tool.

Let's inspect where we're interrupted:

In [9]:
next_task = agent.get_state(config).tasks[0]
next_task.interrupts

(Interrupt(value={'query': 'What should I feed my cat?'}, resumable=True, ns=['agent:5e01f3a3-a097-3cb2-7557-b36fa4c5e427', 'call_tool:154fa07a-05dc-b025-2a4d-e7faa29c643a'], when='during'),)

We can resume execution by issuing a [Command](../../concepts/human_in_the_loop/#the-command-primitive). Note that the data we supply in the `Command` can be customized to your needs based on the implementation of `human_assistance`.

In [10]:
human_response = "You should feed your cat a fish."
human_command = Command(resume={"data": human_response})

for step in agent.stream(human_command, config):
    _print_step(step)


call_tool:
================================= Tool Message =================================

You should feed your cat a fish.

call_model:
================================== Ai Message ==================================

I've reached out for human assistance regarding what to feed your cat, and the suggestion is to feed your cat fish. 

Additionally, the weather in San Francisco is sunny!


Above, when we resume we provide the final tool message, allowing the model to generate its response. Check out the LangSmith traces to see a full breakdown of the runs:

1. [Trace from initial query](https://smith.langchain.com/public/c3d8879d-4d01-41be-807e-6d9eed15df99/r)
2. [Trace after resuming](https://smith.langchain.com/public/97c05ef9-8b4c-428e-8826-3fd417c8c75f/r)

## Review tool calls before execution

To review tool calls before execution, we add a call to [interrupt](../../concepts/human_in_the_loop/#interrupt) to our [entrypoint](../../concepts/functional_api/#entrypoint) implementation. When this function is called, execution will be paused until we issue a command to resume it.

The results of prior tasks-- in this case the initial model call-- are persisted, so that they are not run again following the `interrupt`.

Let's first add a new `review_tool_call` task. Given a tool call, this will `interrupt` for human review. At that point we can either:

- Accept the tool call;
- Revise the tool call and continue;
- Generate a custom tool message (e.g., instructing the model to re-format its tool call).

We will demonstrate these three cases below.

In [2]:
from typing import Union

from langchain_core.messages import ToolCall, ToolMessage
from langgraph.func import entrypoint, task


@task
def review_tool_call(tool_call: ToolCall) -> Union[ToolCall, ToolMessage]:
    """Review a tool call, returning a validated version."""
    human_review = interrupt(
        {
            "question": "Is this correct?",
            "tool_call": tool_call,
        }
    )
    review_action = human_review["action"]
    review_data = human_review.get("data")
    if review_action == "continue":
        return tool_call
    elif review_action == "update":
        updated_tool_call = {**tool_call, **{"args": review_data}}
        return updated_tool_call
    elif review_action == "feedback":
        return ToolMessage(
            content=review_data, name=tool_call["name"], tool_call_id=tool_call["id"]
        )

Otherwise, our tasks are the same. Let's subset to just the `get_weather` tool for this example.

In [3]:
tools = [get_weather]
tools_by_name = {tool.name: tool for tool in tools}


@task
def call_model(messages):
    """Call model with a sequence of messages."""
    response = model.bind_tools(tools).invoke(messages)
    return response


@task
def call_tool(tool_call):
    tool = tools_by_name[tool_call["name"]]
    observation = tool.invoke(tool_call["args"])
    return ToolMessage(content=observation, tool_call_id=tool_call["id"])

Given our tasks, we can update our entrypoint to review the generated tool calls. If a tool call is accepted or revised, we execute in the same way as before. Otherwise, we just append the `ToolMessage` supplied by the human.

In [4]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langgraph.types import Command, interrupt


checkpointer = MemorySaver()

@entrypoint(checkpointer=checkpointer)
def agent(messages, previous):

    if previous is not None:
        messages = add_messages(previous, messages)

    llm_response = call_model(messages).result()
    while True:
        if not llm_response.tool_calls:
            break

        # Review tool calls
        tool_results = []
        tool_calls = []
        for tool_call in llm_response.tool_calls:
            review = review_tool_call(tool_call).result()
            if isinstance(review, ToolMessage):
                tool_results.append(review)
            else:
                tool_calls.append(review)

        # Execute remaining tool calls
        tool_result_futures = [call_tool(tool_call) for tool_call in tool_calls]
        remaining_tool_results = [fut.result() for fut in tool_result_futures]

        # Append to message list
        messages = add_messages(
            messages,
            [llm_response, *tool_results, *remaining_tool_results],
        )

        # Call model again
        llm_response = call_model(messages).result()

    # Generate final response
    messages = add_messages(messages, llm_response)
    return messages

### Usage

Let's demonstrate some scenarios.

In [5]:
def _print_step(step: dict) -> None:
    for task_name, result in step.items():
        if task_name in ("agent", "__interrupt__"):
            continue  # Just print task updates
        else:
            print(f"\n{task_name}:")
            if task_name == "review_tool_call":
                print(result)
            else:
                result.pretty_print()

### Accept a tool call

To accept a tool call, we just indicate in the data we provide in the `Command` that the tool call should pass through.

In [6]:
config = {"configurable": {"thread_id": "1"}}

In [7]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)

{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_sNICg8E5yx8vu8kbJuYdm4Vi)
 Call ID: call_sNICg8E5yx8vu8kbJuYdm4Vi
  Args:
    location: San Francisco


In [8]:
# highlight-next-line
human_input = Command(resume={"action": "continue"})

for step in agent.stream(human_input, config):
    _print_step(step)


review_tool_call:
{'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_sNICg8E5yx8vu8kbJuYdm4Vi', 'type': 'tool_call'}

call_tool:
================================= Tool Message =================================

It's sunny!

call_model:
================================== Ai Message ==================================

The weather in San Francisco is sunny!


### Revise a tool call

To revise a tool call, we can supply updated arguments.

In [9]:
config = {"configurable": {"thread_id": "2"}}

In [10]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)

{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_Yij7KP05J42rulHKlxqTSB2n)
 Call ID: call_Yij7KP05J42rulHKlxqTSB2n
  Args:
    location: san francisco


In [11]:
# highlight-next-line
human_input = Command(resume={"action": "update", "data": {"location": "boston"}})

for step in agent.stream(human_input, config):
    _print_step(step)


review_tool_call:
{'name': 'get_weather', 'args': {'location': 'boston'}, 'id': 'call_Yij7KP05J42rulHKlxqTSB2n', 'type': 'tool_call'}

call_tool:
================================= Tool Message =================================

It's rainy!

call_model:
================================== Ai Message ==================================

The weather in San Francisco is rainy.


### Generate a custom ToolMessage

To Generate a custom `ToolMessage`, we supply the content of the message. In this case we will ask the model to reformat its tool call.

In [12]:
config = {"configurable": {"thread_id": "3"}}

In [13]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)

{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_XQtgUchIAkMeHUYAwtjvN7pg)
 Call ID: call_XQtgUchIAkMeHUYAwtjvN7pg
  Args:
    location: san francisco


In [14]:
# highlight-next-line
human_input = Command(
    # highlight-next-line
    resume={
        # highlight-next-line
        "action": "feedback",
        # highlight-next-line
        "data": "Please format as <City>, <State>.",
    # highlight-next-line
    },
# highlight-next-line
)

for step in agent.stream(human_input, config):
    _print_step(step)


review_tool_call:
content='Please format as <City>, <State>.' name='get_weather' id='76e420fd-301a-400d-a31c-41f43ee865a9' tool_call_id='call_rDf4c8Zsw1JzyLTrFcdl61bC'

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_vVsfdFZXisifRA9JliZKqHau)
 Call ID: call_vVsfdFZXisifRA9JliZKqHau
  Args:
    location: San Francisco, CA

review_tool_call:
content='Please format as <City>, <State>.' name='get_weather' tool_call_id='call_vVsfdFZXisifRA9JliZKqHau'

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_iarrPN17agVG6NByasbEC2kO)
 Call ID: call_iarrPN17agVG6NByasbEC2kO
  Args:
    location: San Francisco, California

review_tool_call:
content='Please format as <City>, <State>.' name='get_weather' tool_call_id='call_iarrPN17agVG6NByasbEC2kO'

call_model:
================================== Ai Message ==================================
Tool Calls:


KeyboardInterrupt



Once it is re-formatted, we can accept it:

In [15]:
# highlight-next-line
human_input = Command(resume={"action": "continue"})

for step in agent.stream(human_input, config):
    _print_step(step)


call_tool:
================================= Tool Message =================================

It's sunny!

call_model:
================================== Ai Message ==================================

The weather in San Francisco, CA is sunny!
